In [8]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta
from scipy.optimize import minimize

const days_in_a_year = 252
# Function to download stock data
def download_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    stock_data['Close'].to_csv(f"new_{symbol}.csv")
    return stock_data

# Function to load stock data from file or download if not available
def get_stock_data(symbols):
    end_date = datetime.today()
    start_date = end_date - timedelta(days=30*365)
    
    stock_data_dict = pd.DataFrame()
    
    for symbol in symbols:
        file_name = f"new_{symbol}.csv"
        if os.path.exists(file_name):
            stock_data = pd.read_csv(file_name, index_col='Date', parse_dates=True)
        else:
            stock_data = download_stock_data(symbol, start_date, end_date)
        stock_data_dict[symbol] = stock_data['Close']
    
    return pd.DataFrame(stock_data_dict)

# Function to calculate portfolio performance metrics
def portfolio_performance(weights, mean_returns, cov_matrix):
    returns = np.sum(mean_returns * weights) * 252
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    return returns, std_dev

# Function to calculate negative Sharpe Ratio
def neg_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    p_returns, p_std_dev = portfolio_performance(weights, mean_returns, cov_matrix)
    return -(p_returns - risk_free_rate) / p_std_dev

# Function to optimize portfolio for maximum Sharpe Ratio
def optimize_portfolio(mean_returns, cov_matrix):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bounds = tuple((0, 1) for asset in range(num_assets))
    
    result = minimize(neg_sharpe_ratio, num_assets*[1./num_assets], args=args,
                      method='SLSQP', bounds=bounds, constraints=constraints)
    
    return result

# Function to calculate rolling window portfolio metrics
def rolling_window_portfolio(stock_data):
    window_size = 3 * 252  # 3 years of trading days
    step_size = 252  # 1 year of trading days
    
    results = []
    
    for start in range(0, len(stock_data) - window_size + 1, step_size):
        end = start + window_size
        window_data = stock_data.iloc[start:end]
        
        mean_returns = window_data.pct_change().mean()
        cov_matrix = window_data.pct_change().cov()
        
        optimized_result = optimize_portfolio(mean_returns, cov_matrix)
        
        weights = optimized_result.x
        returns, std_dev = portfolio_performance(weights, mean_returns, cov_matrix)
        
        drawdown = (window_data / window_data.cummax() - 1).min().min()
        max_profit = (window_data.iloc[-1] / window_data.iloc[0] - 1).max()
        sharpe_ratio = -optimized_result.fun
        
        results.append({
            'Start Date': window_data.index[0],
            'End Date': window_data.index[-1],
            'Return': returns,
            'Volatility': std_dev,
            'Drawdown': drawdown,
            'Max Profit': max_profit,
            'Sharpe Ratio': sharpe_ratio
        })
    
    return pd.DataFrame(results)

# List of stock symbols
symbols = ['AAPL', 'MSFT', 'GOOGL']

# Get stock data
stock_data = get_stock_data(symbols)

# Calculate rolling window portfolio metrics
portfolio_metrics_df = rolling_window_portfolio(stock_data)

# Save the results to a CSV file
portfolio_metrics_df.to_csv('portfolio_metrics.csv', index=False)

print("Portfolio metrics have been calculated and saved to portfolio_metrics.csv")



SyntaxError: invalid syntax (3267363725.py, line 8)